# Route Checker

## 1. Batfishコンテナの起動

Batfishコンテナが起動しているか確認し、起動していない場合は起動する。

In [1]:
%%bash
if (docker ps -f "ancestor=batfish/batfish" | grep "0.0.0.0:9996-9997" > /dev/null); then
  echo "Batfish container is already running."
else
  docker run -d -p 9997:9997 -p 9996:9996 batfish/batfish
fi

bfcfcba996746b9171a81b681f7e4d8098b96e2e83a9b64a709392fbcf684d6d


## 2. startup.pyの実行

startup.pyを実行し、ライブラリのインポートなどを行う。

In [4]:
%run startup.py

## 3. 通信の設定

以下のセル内の各変数に検証したい通信の情報を格納し、実行する。
HeaderConstraintsのその他の引数は以下を参照。
[https://github.com/batfish/pybatfish/blob/master/pybatfish/datamodel/flow.py#L647-L669](htps://github.com/batfish/pybatfish/blob/master/pybatfish/datamodel/flow.py#L647-L669)

In [5]:
# 送信元IPアドレス
src_ip = "192.168.10.1"
# 宛先IPアドレス
dst_ip = "192.168.20.1"
# トランスポートプロトコル
ip_protocol = "TCP"
# 送信元ポート番号
src_port ="49512"
# 宛先ポート番号
dst_port = "80"

outbound_flow = HeaderConstraints(srcIps=src_ip,
                         dstIps=dst_ip,
                         ipProtocols=ip_protocol,
                         srcPorts=src_port,
                         dstPorts=dst_port,
                        )
return_flow = HeaderConstraints(srcIps=dst_ip,
                         dstIps=src_ip,
                         ipProtocols=ip_protocol,
                         srcPorts=dst_port,
                         dstPorts=src_port,
                         tcpFlags=[MatchTcpFlags.match_ack()]
                         )

## 4 ホストの設定

以下のセル内のJSONに送信元および宛先のホスト情報を指定し、実行する。

In [6]:
import json

host_file_dir = "networks/example2/hosts/"
src_host_dict = {
                 "hostname": "src_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(src_ip + "/24"),
                     }
                   }
                 }

dst_host_dict = {
                 "hostname": "dst_host",
                 "hostInterfaces": {
                   "eth0": {
                     "name": "eth0",
                     "prefix": str(dst_ip + "/24"),
                     }
                   }
                 }

f = open("{dir}src_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(src_host_dict))
f.close()

f = open("{dir}dst_host.json".format(dir=host_file_dir), "w")
f.write(json.dumps(dst_host_dict))
f.close()

## 3. 設定ファイルのアップロード

設定ファイルをBatfishコンテナへアップロードする(スナップショット名の後にいくつかのログが表示される)。

In [23]:
# ネットワークの名前を指定する
NETWORK_NAME = "datacenter1"
# ネットワークの状態の名前を指定する(current, candidateなど)
SNAPSHOT_NAME = "current"
# 設定ファイルの格納先を指定する
SNAPSHOT_PATH = "networks/example-route-checker"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'current'

## 5. 通信の検証

以下のセルを実行し、結果を確認する。

In [19]:
# 往路
traceroute_ob_ans = bfq.traceroute(
                                headers=outbound_flow,
                                startLocation="src_host",
                                #startLocation="enter(core[GigabitEthernet1/0])"
                               ).answer()
display_html(traceroute_ob_ans.frame())

,Flow,Traces,TraceCount
0,Start Location: src_hostSrc IP: 192.168.10.1Src Port: 49512Dst IP: 192.168.20.1Dst Port: 80IP Protocol: TCP,"ACCEPTED1. node: src_host ORIGINATED(default) FORWARDED(Routes: static [Network: 0.0.0.0/0, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(eth0)2. node: core RECEIVED(GigabitEthernet1/0: seg10in) FORWARDED(Routes: connected [Network: 192.168.20.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)3. node: dst_host RECEIVED(eth0) ACCEPTED(InboundStep)",1


In [24]:
# 復路
traceroute_rt_ans = bfq.traceroute(
                                   headers=return_flow,
                                   startLocation="dst_host",
                                  ).answer()
display_html(traceroute_rt_ans.frame())

,Flow,Traces,TraceCount
0,Start Location: dst_hostSrc IP: 192.168.20.1Src Port: 80Dst IP: 192.168.10.1Dst Port: 49512IP Protocol: TCP,"DENIED_IN1. node: dst_host ORIGINATED(default) FORWARDED(Routes: static [Network: 0.0.0.0/0, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(eth0)2. node: core DENIED(GigabitEthernet2/0: seg20in)",1
